In [37]:
# Improved Tier Classification — full cell to paste into your notebook
# Replace DATA_PATH if the provided path isn't your dataset CSV.
DATA_PATH = "indian_pharmaceutical_products_clean.csv"  # update if needed

import os, re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses, callbacks
from tensorflow.keras.layers import TextVectorization

RND = 42
np.random.seed(RND)
tf.random.set_seed(RND)

# -----------------------
# 0. Load data
# -----------------------
print("Loading:", DATA_PATH)
df = pd.read_csv(DATA_PATH)
print("Rows:", len(df))
print("Columns:", df.columns.tolist())

# Keep only relevant columns (you listed these earlier); adjust if names differ
cols_keep = ['product_id','brand_name','manufacturer','price_inr','dosage_form','pack_size','pack_unit',
             'num_active_ingredients','primary_ingredient','primary_strength','active_ingredients','therapeutic_class']
use_cols = [c for c in cols_keep if c in df.columns]
df = df[use_cols].copy()
print("Using columns:", use_cols)

# -----------------------
# 1. Lightweight cleaning & parsing
# -----------------------
def normalize_text(s):
    s = str(s).lower()
    s = re.sub(r'[\(\)\[\]\{\},;:/\\\|"]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    s = re.sub(r'(\d+)\s*mg', r'\1mg', s)
    s = re.sub(r'(\d+)\s*ml', r'\1ml', s)
    return s

def parse_strength(s):
    s = str(s).lower()
    m = re.search(r'(\d+(?:\.\d+)?)\s*(mcg|mg|g|µg|iu)?', s)
    if not m: return np.nan
    v = float(m.group(1)); unit = (m.group(2) or '').replace('µg','mcg')
    if unit == 'mcg': return v/1000.0
    if unit == 'g': return v*1000.0
    return v

def parse_pack(size, unit):
    try:
        if not pd.isna(size): return int(size)
    except: pass
    m = re.search(r'(\d+)', str(unit).lower())
    return int(m.group(1)) if m else np.nan

# Fill/convert safely
df['brand_name'] = df.get('brand_name','').fillna('').astype(str)
df['manufacturer'] = df.get('manufacturer','').fillna('unknown').astype(str)
df['primary_ingredient'] = df.get('primary_ingredient','').fillna('').astype(str)
df['active_ingredients'] = df.get('active_ingredients','').fillna('').astype(str)
df['primary_strength'] = df.get('primary_strength','').fillna('').astype(str)
df['dosage_form'] = df.get('dosage_form','').fillna('').astype(str)
df['pack_size'] = pd.to_numeric(df.get('pack_size', pd.NA), errors='coerce')

# parsed fields
df['brand_clean'] = df['brand_name'].apply(normalize_text)
df['primary_ing_clean'] = df['primary_ingredient'].apply(normalize_text)
df['active_ing_clean'] = df['active_ingredients'].apply(normalize_text)
df['strength_mg'] = df['primary_strength'].apply(parse_strength)
df['pack_num'] = df.apply(lambda r: parse_pack(r.get('pack_size', pd.NA), r.get('pack_unit','')), axis=1)

# safe fills
df['pack_num'] = df['pack_num'].fillna(df['pack_num'].median())
df['strength_mg'] = df['strength_mg'].fillna(df['strength_mg'].median())

# combined text
df['composition_text'] = (df['primary_ing_clean'] + ' ' + df['active_ing_clean']).str.strip()
df['text_for_emb'] = (df['brand_clean'] + ' || ' + df['composition_text'] + ' || ' + df['dosage_form'].str.lower()).str.strip()

# -----------------------
# 2. Build price_tier (target) if not present
# -----------------------
if 'price_inr' not in df.columns:
    raise SystemExit("price_inr required for generating tiers. Provide a dataset with price_inr.")
df = df[df['price_inr'].notna()].reset_index(drop=True)
df['price_tier'] = pd.qcut(df['price_inr'], q=3, labels=[0,1,2]).astype(int)

# -----------------------
# 3. Embeddings (SBERT recommended) with fallback
# -----------------------
USE_SBERT = True
emb = None
try:
    if USE_SBERT:
        from sentence_transformers import SentenceTransformer
        print("Using SBERT embeddings (all-MiniLM-L6-v2).")
        sbert = SentenceTransformer('all-MiniLM-L6-v2')
        emb = sbert.encode(df['text_for_emb'].astype(str).tolist(), batch_size=128, show_progress_bar=True)
except Exception:
    print("SBERT not available or failed — falling back to Keras TextVectorization + BiLSTM encoder.")
    USE_SBERT = False

if not USE_SBERT:
    MAX_VOCAB = 10000; SEQ_LEN = 32; EMBED_DIM = 64; LSTM_UNITS = 64
    texts = df['text_for_emb'].astype(str).values
    vectorizer = TextVectorization(max_tokens=MAX_VOCAB, output_sequence_length=SEQ_LEN)
    vectorizer.adapt(texts)
    # small encoder
    inp = layers.Input(shape=(1,), dtype=tf.string)
    x = vectorizer(inp)
    x = layers.Embedding(input_dim=len(vectorizer.get_vocabulary()), output_dim=EMBED_DIM, mask_zero=True)(x)
    x = layers.Bidirectional(layers.LSTM(LSTM_UNITS))(x)
    x = layers.Dense(64, activation='relu')(x)
    enc = models.Model(inp, x)
    emb = enc.predict(texts, batch_size=256, verbose=1)

df['embedding'] = list(emb)
EMB_DIM = emb.shape[1]

# -----------------------
# 4. Engineer features helpful for tier classification
# -----------------------
# manufacturer group & stats
le_man = LabelEncoder()
manu_counts = df['manufacturer'].value_counts()
rare_manu = manu_counts[manu_counts <= 10].index
df['manu_group'] = df['manufacturer'].apply(lambda x: 'other' if x in rare_manu else x)
df['manu_id'] = le_man.fit_transform(df['manu_group'].astype(str))

# composition key & competition count
df['composition_key'] = df['primary_ing_clean'].astype(str) + '||' + df['strength_mg'].astype(str)
df['comp_count'] = df.groupby('composition_key')['product_id'].transform('count')

# cluster: PCA on embeddings -> kmeans minibatch to get cheap cluster id
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
pca = PCA(n_components=16, random_state=RND)
emb_pca = pca.fit_transform(np.vstack(df['embedding'].values))
for i in range(emb_pca.shape[1]):
    df[f'emb_pca_{i}'] = emb_pca[:, i]
mbk = MiniBatchKMeans(n_clusters=min(150, max(10, df.shape[0]//1000)), batch_size=4096, random_state=RND)
df['equivalence_cluster'] = mbk.fit_predict(emb_pca)

# cluster-level price medians (weak but useful)
df['cluster_price_med'] = df.groupby('equivalence_cluster')['price_inr'].transform('median')
df['manu_price_med'] = df.groupby('manu_group')['price_inr'].transform('median')

# unit price
df['cost_per_unit'] = df['price_inr'] / (df['pack_num'] + 1e-9)

# create small categorical features that help classification
df['dosage_form_cat'] = df['dosage_form'].astype(str).fillna('unknown')
le_dos = LabelEncoder(); df['dosage_id'] = le_dos.fit_transform(df['dosage_form_cat'])

# -----------------------
# 5. Prepare train/test splits and numeric scaling
# -----------------------
features_numeric = ['pack_num','strength_mg','comp_count','cluster_price_med','manu_price_med','cost_per_unit'] + [f'emb_pca_{i}' for i in range(emb_pca.shape[1])]
num_scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[features_numeric] = num_scaler.fit_transform(df_scaled[features_numeric].fillna(0))

# train/val/test stratified by tier
train_df, temp_df = train_test_split(df_scaled, test_size=0.25, random_state=RND, stratify=df_scaled['price_tier'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=RND, stratify=temp_df['price_tier'])
print("sizes:", len(train_df), len(val_df), len(test_df))

# -----------------------
# 6. Build model inputs (embeddings and categorical embeddings)
# -----------------------
def make_inputs(d):
    return {
        'emb_input': np.vstack(d['embedding'].values).astype('float32'),
        'manu_input': d['manu_id'].astype('int32').values,
        'dosage_input': d['dosage_id'].astype('int32').values,
        'num_input': d[features_numeric].astype('float32').values
    }

X_train = make_inputs(train_df)
y_train = train_df['price_tier'].astype('int32').values
X_val = make_inputs(val_df)
y_val = val_df['price_tier'].astype('int32').values
X_test = make_inputs(test_df)
y_test = test_df['price_tier'].astype('int32').values

# -----------------------
# 7. Compute class weights
# -----------------------
classes = np.unique(y_train)
cw = class_weight.compute_class_weight('balanced', classes=classes, y=y_train)
class_weight_dict = dict(enumerate(cw))
print("Class weights:", class_weight_dict)

# -----------------------
# 8. Build improved classification NN
# -----------------------
# Inputs
emb_in = layers.Input(shape=(EMB_DIM,), name='emb_input')
manu_in = layers.Input(shape=(), dtype='int32', name='manu_input')
dosage_in = layers.Input(shape=(), dtype='int32', name='dosage_input')
num_in = layers.Input(shape=(len(features_numeric),), dtype='float32', name='num_input')

# categorical embeddings
MANU_VOCAB = df_scaled['manu_id'].nunique() + 2
MANU_EMB_DIM = min(32, max(8, MANU_VOCAB//10))
DOSAGE_VOCAB = df_scaled['dosage_id'].nunique() + 2
DOSAGE_EMB_DIM = min(8, max(4, DOSAGE_VOCAB//4))

manu_emb = layers.Embedding(input_dim=MANU_VOCAB, output_dim=MANU_EMB_DIM, name='manu_emb')(manu_in)
manu_emb = layers.Flatten()(manu_emb)
dos_emb = layers.Embedding(input_dim=DOSAGE_VOCAB, output_dim=DOSAGE_EMB_DIM, name='dos_emb')(dosage_in)
dos_emb = layers.Flatten()(dos_emb)

# concat
x = layers.Concatenate()([emb_in, manu_emb, dos_emb, num_in])

# a slightly deeper block with batchnorm & dropout
x = layers.Dense(256, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.2)(x)

out = layers.Dense(3, activation='softmax', name='tier')(x)

tier_model = models.Model(inputs=[emb_in, manu_in, dosage_in, num_in], outputs=out)
# Prioritize classification: optimizer + lr
tier_model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
tier_model.summary()

# -----------------------
# 9. Train with callbacks (monitor val accuracy and val macro-f1 via callback)
# -----------------------
# Macro-F1 is not a built-in Keras metric; we'll compute on val at epoch end using a callback.
from sklearn.metrics import f1_score

class ValMetricsCallback(callbacks.Callback):
    def __init__(self, val_data, batch_size=1024):
        super().__init__()
        self.x_val, self.y_val = val_data
        self.batch_size = batch_size
    def on_epoch_end(self, epoch, logs=None):
        preds = self.model.predict(self.x_val, batch_size=self.batch_size, verbose=0)
        preds_cls = np.argmax(preds, axis=1)
        mac_f1 = f1_score(self.y_val, preds_cls, average='macro')
        logs = logs or {}
        logs['val_macro_f1'] = mac_f1
        print(f" — val_macro_f1: {mac_f1:.4f}")

val_cb = ValMetricsCallback(((X_val['emb_input'], X_val['manu_input'], X_val['dosage_input'], X_val['num_input']), y_val), batch_size=1024)
es = callbacks.EarlyStopping(monitor='val_macro_f1', mode='max', patience=6, restore_best_weights=True, verbose=1)
rlr = callbacks.ReduceLROnPlateau(monitor='val_macro_f1', mode='max', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
mc = callbacks.ModelCheckpoint('tier_nn_improved.h5', monitor='val_macro_f1', mode='max', save_best_only=True, verbose=1)

# Note Keras doesn't accept class_weight for a single-output sparse_categorical if you pass dict in older TF versions.
# We pass class_weight mapping directly to fit
history = tier_model.fit(
    x = {'emb_input': X_train['emb_input'], 'manu_input': X_train['manu_input'],
         'dosage_input': X_train['dosage_input'], 'num_input': X_train['num_input']},
    y = y_train,
    validation_data = ({'emb_input': X_val['emb_input'], 'manu_input': X_val['manu_input'],
                        'dosage_input': X_val['dosage_input'], 'num_input': X_val['num_input']}, y_val),
    epochs = 60,
    batch_size = 256,
    class_weight = class_weight_dict,
    callbacks = [val_cb, es, rlr, mc],
    verbose = 2
)

# -----------------------
# 10. Evaluate on test set
# -----------------------
preds_test = tier_model.predict({'emb_input': X_test['emb_input'], 'manu_input': X_test['manu_input'],
                                 'dosage_input': X_test['dosage_input'], 'num_input': X_test['num_input']}, batch_size=1024)
preds_cls = np.argmax(preds_test, axis=1)
acc = accuracy_score(y_test, preds_cls)
mac_f1 = f1_score(y_test, preds_cls, average='macro')
print("\nTest accuracy:", acc)
print("Test macro F1:", mac_f1)
print("\nClassification report:")
print(classification_report(y_test, preds_cls, digits=4))
print("\nConfusion matrix:\n", confusion_matrix(y_test, preds_cls))

# Save final model
tier_model.save('tier_nn_improved.h5')
print("Saved model -> tier_nn_improved.h5")


Loading: indian_pharmaceutical_products_clean.csv
Rows: 253973
Columns: ['product_id', 'brand_name', 'manufacturer', 'price_inr', 'is_discontinued', 'dosage_form', 'pack_size', 'pack_unit', 'num_active_ingredients', 'primary_ingredient', 'primary_strength', 'active_ingredients', 'therapeutic_class', 'packaging_raw', 'manufacturer_raw']
Using columns: ['product_id', 'brand_name', 'manufacturer', 'price_inr', 'dosage_form', 'pack_size', 'pack_unit', 'num_active_ingredients', 'primary_ingredient', 'primary_strength', 'active_ingredients', 'therapeutic_class']
SBERT not available or failed — falling back to Keras TextVectorization + BiLSTM encoder.
993/993 ━━━━━━━━━━━━━━━━━━━━ 46s 46ms/step
sizes: 190479 31747 31747
Class weights: {0: 0.9921401337583599, 1: 0.9935840257890866, 2: 1.014589325663151}


Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ manu_input (InputLayer)       │ (None)                    │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dosage_input (InputLayer)     │ (None)                    │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ manu_emb (Embedding)          │ (None, 32)                │         121,760 │ manu_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dos_emb (Embedding)           │ (None, 4)                 │              68 │ dosage_input[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ emb_input (InputLayer)        │ (None, 64)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_8 (Flatten)           │ (None, 32)                │               0 │ manu_emb[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_9 (Flatten)           │ (None, 4)                 │               0 │ dos_emb[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ num_input (InputLayer)        │ (None, 22)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_9 (Concatenate)   │ (None, 122)               │               0 │ emb_input[0][0],           │
│                               │                           │                 │ flatten_8[0][0],           │
│                               │                           │                 │ flatten_9[0][0],           │
│                               │                           │                 │ num_input[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_41 (Dense)              │ (None, 256)               │          31,488 │ concatenate_9[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 256)               │           1,024 │ dense_41[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_8 (Dropout)           │ (None, 256)               │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_42 (Dense)              │ (None, 128)               │          32,896 │ dropout_8[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 128)               │             512 │ dense_42[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_9 (Dropout)           │ (None, 128)               │               

 Total params: 196,199 (766.40 KB)

 Trainable params: 195,431 (763.40 KB)

...

745/745 - 5s - 7ms/step - accuracy: 0.9326 - loss: 0.1648 - val_accuracy: 0.9416 - val_loss: 0.1431 - val_macro_f1: 0.9419 - learning_rate: 5.0000e-05
Epoch 50/60
 — val_macro_f1: 0.9411

Epoch 50: val_macro_f1 did not improve from 0.94188
745/745 - 5s - 6ms/step - accuracy: 0.9337 - loss: 0.1632 - val_accuracy: 0.9408 - val_loss: 0.1458 - val_macro_f1: 0.9411 - learning_rate: 5.0000e-05
Epoch 51/60
 — val_macro_f1: 0.9412

Epoch 51: val_macro_f1 did not improve from 0.94188
745/745 - 5s - 7ms/step - accuracy: 0.9339 - loss: 0.1619 - val_accuracy: 0.9409 - val_loss: 0.1445 - val_macro_f1: 0.9412 - learning_rate: 5.0000e-05
Epoch 52/60
 — val_macro_f1: 0.9426

Epoch 52: val_macro_f1 improved from 0.94188 to 0.94261, saving model to tier_nn_improved.h5


745/745 - 6s - 8ms/step - accuracy: 0.9348 - loss: 0.1594 - val_accuracy: 0.9423 - val_loss: 0.1418 - val_macro_f1: 0.9426 - learning_rate: 5.0000e-05
Epoch 53/60
 — val_macro_f1: 0.9414

Epoch 53: val_macro_f1 did not improve from 0.94261
745/745 - 5s - 7ms/step - accuracy: 0.9350 - loss: 0.1586 - val_accuracy: 0.9411 - val_loss: 0.1425 - val_macro_f1: 0.9414 - learning_rate: 5.0000e-05
Epoch 54/60
 — val_macro_f1: 0.9420

Epoch 54: val_macro_f1 did not improve from 0.94261
745/745 - 7s - 9ms/step - accuracy: 0.9360 - loss: 0.1568 - val_accuracy: 0.9417 - val_loss: 0.1447 - val_macro_f1: 0.9420 - learning_rate: 5.0000e-05
Epoch 55/60
 — val_macro_f1: 0.9425

Epoch 55: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.

Epoch 55: val_macro_f1 did not improve from 0.94261
745/745 - 6s - 8ms/step - accuracy: 0.9360 - loss: 0.1558 - val_accuracy: 0.9422 - val_loss: 0.1413 - val_macro_f1: 0.9425 - learning_rate: 5.0000e-05
Epoch 56/60
 — val_macro_f1: 0.9439

Epoch 56: val_

745/745 - 6s - 8ms/step - accuracy: 0.9395 - loss: 0.1495 - val_accuracy: 0.9436 - val_loss: 0.1387 - val_macro_f1: 0.9439 - learning_rate: 2.5000e-05
Epoch 57/60
 — val_macro_f1: 0.9436

Epoch 57: val_macro_f1 did not improve from 0.94389
745/745 - 5s - 7ms/step - accuracy: 0.9390 - loss: 0.1488 - val_accuracy: 0.9433 - val_loss: 0.1378 - val_macro_f1: 0.9436 - learning_rate: 2.5000e-05
Epoch 58/60
 — val_macro_f1: 0.9430

Epoch 58: val_macro_f1 did not improve from 0.94389
745/745 - 5s - 7ms/step - accuracy: 0.9397 - loss: 0.1480 - val_accuracy: 0.9426 - val_loss: 0.1429 - val_macro_f1: 0.9430 - learning_rate: 2.5000e-05
Epoch 59/60
 — val_macro_f1: 0.9438

Epoch 59: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-05.

Epoch 59: val_macro_f1 did not improve from 0.94389
745/745 - 5s - 7ms/step - accuracy: 0.9407 - loss: 0.1471 - val_accuracy: 0.9435 - val_loss: 0.1391 - val_macro_f1: 0.9438 - learning_rate: 2.5000e-05
Epoch 60/60
 — val_macro_f1: 0.9452

Epoch 60: val_

745/745 - 5s - 7ms/step - accuracy: 0.9406 - loss: 0.1456 - val_accuracy: 0.9449 - val_loss: 0.1360 - val_macro_f1: 0.9452 - learning_rate: 1.2500e-05
Restoring model weights from the end of the best epoch: 60.
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step



Test accuracy: 0.9463886351466281
Test macro F1: 0.9467127559302352

Classification report:
              precision    recall  f1-score   support

           0     0.9615    0.9316    0.9463     10666
           1     0.9069    0.9374    0.9219     10651
           2     0.9732    0.9708    0.9720     10430

    accuracy                         0.9464     31747
   macro avg     0.9472    0.9466    0.9467     31747
weighted avg     0.9470    0.9464    0.9465     31747


Confusion matrix:
 [[ 9936   725     5]
 [  393  9984   274]
 [    5   300 10125]]
Saved model -> tier_nn_improved.h5
